# Decorators...

Decorators are mainly used to either modify a callable or do something with it when and where it's defined, they look like this:
```python
@some_decorator
def my_func():
    ...
```

Understanding how decorators work _really_ helps me understand how, why, and when to use them, and enables me to write my own

# ...How do they work??

#### Spoiler: They're **not** magic

#### Well, maybe just a little bit...

#### Nope, just some syntactic sugar, not really magic.

**Syntactic sugar**: special language features to make an already available pattern easier to write and/or clearer to read for humans

#### They are _**VERY**_ abstract, though...

## What are "callables"?  Why make up words?

- "A callable is anything that can be called" -- [top StackOverflow answer][1]; true, but not very helpful
- Why "callable"?
    - Many distinct kinds; the term "function" has baggage
- _But_ for this talk I'll just be using functions
[1]: https://stackoverflow.com/questions/111234/what-is-a-callable-in-python#111255

### "Snippets of code"

- Easy to take for granted, but very powerful:
    - Can operate on data - take parameters, return results
    - Callables can call other callables

### Code as data

- callbacks, helpers (e.g. `sort`'s `key=`)
- "inner functions"

### Callables can take callables...

In [1]:
def hello(printer):
    printer("Hello world")

hello(print)

Hello world


In [2]:
def exclamation(s):
    print(f"{s}!")

hello(exclamation)

Hello world!


## ...and return callables!

In [3]:
def greeter(printer):
    def greet(name):
        # Inner function has - and KEEPS - the context in 
        # which is was created ("closes over", aka "closure")
        printer(f"Hello {name}")
    return greet

hi = greeter(print)
hi_bang = greeter(exclamation)

hi("Python Study Group")
hi_bang("Python Study Group")

Hello Python Study Group
Hello Python Study Group!


## Let's try this again

In [10]:
from random import choice, random
def flaky():
    return choice(["Okay", None])
    
flaky()

'Okay'

In [11]:
def retry(fn):
    tries = 5
    for t in range(tries):
        result = fn()
        if result:
            return result
        print(f"fail {t}")
    return result

retry(flaky)

fail 0
fail 1


'Okay'

In [12]:
def retried(fn):
    tries = 5
    def retry_wrapper():
        for t in range(tries):
            result = fn()
            if result:
                return result
            print(f"fail {t}")
        return result
    return retry_wrapper

flaky = retried(flaky)
flaky()

fail 0
fail 1
fail 2
fail 3


'Okay'

## Add a little syntactic sugar...

In [18]:
@retried
def flaky():
    return choice(["Okay", None])

flaky()

fail 0
fail 1


'Okay'

## How does that work?

**retried**: expression that _<u>takes</u>_ a callable and _<u>returns</u>_ a callable that _<u>replaces<u>_ the following one

In [19]:
retried

<function __main__.retried(fn)>

**@**: the sugar, automatically calls the decorator _<u>at definition time</u>_ with the following one and does the replacement, basically like we did above.

## Getting fancy

Let's make a version with configurable tries, say, `retry_n`...

...but wait, if the `@` automatically calls it with the following thing - _<u>just</u>_ the following thing - where would arguments for _<u>it</u>_ go?

## Inception

### Callables can return _<u>decorators!</u>_

In [20]:
def retry_n(tries):
    def retried(fn):
        def retry_wrapper():
            for t in range(tries):
                result = fn()
                if result:
                    return result
                print(f"fail {t}")
            return result
        return retry_wrapper
    return retried

@retry_n(2)
def best_effort():
    return choice(["Okay", None])

In [37]:
best_effort()

fail 0
fail 1


### What's going on here?
```python
def best_effort():
    return choice(["Okay", None])
best_effort = retry_n(2)(best_effort)
```

## Multiple decorators

In [38]:
def bold(fn):
    def stars():
        return(f"*{fn()}*")
    return stars

def italic(fn):
    def unders():
        return(f"_{fn()}_")
    return unders

In [39]:
@bold
@italic
def hi():
    return "Hello"

hi()
# Will it be '_*Hello*_' or '*_Hello_*'?

'*_Hello_*'

```python
hi = bold(italic(hi))
```

In [40]:
@italic
@bold
def hi():
    return "Hello"

hi()

'_*Hello*_'

```python
hi = italic(bold(hi))
```

## Professional touches

In [41]:
@retried
def flaky_greet(name, reliablility=0.5):
    "Greet by name with adjustable reliablility."
    return f"Hello {name}" if random() <= reliablility else None

In [42]:
help(flaky_greet)

Help on function retry_wrapper in module __main__:

retry_wrapper()



In [43]:
flaky_greet("Ben")

TypeError: retry_wrapper() takes 0 positional arguments but 1 was given

In [45]:
import functools
def retried(f):
    tries = 5
    @functools.wraps(f)
    def retry_wrapper(*args, **kwargs):
        for t in range(tries):
            result = f(*args, **kwargs)
            if result:
                return result
            print(f"fail {t}")
        return result
    return retry_wrapper

In [46]:
@retried
def flaky_greet(name, reliablility=0.5):
    "Greet by name with adjustable reliablility."
    return f"Hello {name}" if random() <= reliablility else None

In [47]:
help(flaky_greet)

Help on function flaky_greet in module __main__:

flaky_greet(name, reliablility=0.5)
    Greet by name with adjustable reliablility.



In [51]:
flaky_greet("Ben")

fail 0
fail 1
fail 2


'Hello Ben'

In [52]:
flaky_greet("Ben", reliablility=0.95)

'Hello Ben'

## Uses for decorators

- Change the behavior of a callable (e.g. retry, lru_cache, ddt)
- Change the behavior of _something else_ "during" a callable
- Extract boilerplate from the start and end of functions
    - Complimentary to "extract function"
- "Registering" callables
    - Often returns original
- Implementation swap
    - Avoid re-checking invariant condition (could also use if/else around definitions)
- ...and more!
- There are also "class decorators"
    - "Takes a class and returns a class to replace the original"
    - Not the same thing as classes that vend decorators, which are very useful

In [33]:
from functools import lru_cache
import shutil
import subprocess as sub
import sys
def tput_lookup(*cap):
    "Get the value for a capname from tput, see man tput for more information"
    tp_cmd = ["tput",] + [ str(i) for i in cap ]
    result = sub.run(tp_cmd, stdout=sub.PIPE)
    r = ""  # Default to falsey
    if result.returncode == 0:  # If success, capability exists or property is true
        r = result.stdout.decode("utf-8")
        if r:
            try:
                r = int(r)  # Note: many of the int
            except ValueError:
                pass  # Nope, not an int
        else:
            r = True  # tput command success means "cap is true"
    return r

In [34]:
def tput_swap(dummy):
    "Decorator to replace a dummy implementation with calls to tput if appropreate and available"
    if sys.stdout.isatty() and shutil.which("tput"):
        return lru_cache()(tput_lookup)  # Note: some attributes (e.g. "cols") can change, so cache may be stale
    else:
        return dummy


@tput_swap
def tput(*cap):
    "Dummy tput implementation, just returns empty string for now"
    return ""  # This might not make much sense for some things (eg "cols"), so callers may need to test and use sensible defaults